In [1]:
import os
os.environ['JAVA_HOME'] = "/home/yh/anaconda3/envs/webshop/bin/"
os.environ['JVM_PATH'] = "/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so"

In [3]:
import os

# Change to a directory
os.chdir('/home/yh/LLMAgent/webshop/baseline_models')


from train_rl import parse_args as webenv_args
import imp
from env import WebEnv 


args = webenv_args()[0]
env = WebEnv(args, split='test')
print('env loaded')

Products loaded.

Keys cleaned.

Attributes loaded.

100%|████████████████████████████████████████████████████████████████| 1181436/1181436 [00:14<00:00, 83535.80it/s]


164 skipped

Loaded 12087 goals.
range(0, 500)


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


env loaded
